In [17]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

In [18]:
# Call the graph the exact same thing as its notebook (minus the ipynb suffix)
GRAPH_NAME = "reserve_demand"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

In [19]:
pio.templates.default = 'fed_2025'

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

In [20]:
GRAPH_OUTPUT_PATH

WindowsPath('C:/Users/Bobby/Desktop/Fed Challenge 2025/NEUFed2025/figures')

In [61]:
today = dt.date.today()

hlb_df = pd.read_excel(ut.get_repo_root() / "code" / "data" / "reserves_2.xlsx").set_index('Date')
hlb_df = hlb_df.rename(columns={
    'Elasticity - 50th percentile (main)': '50th percentile',
    'Elasticity - 16th percentile': '16th percentile',
    'Elasticity - 84th percentile': '84th percentile',
})

hlb_df



,50th percentile,Elasticity - 2.5th percentile,Elasticity - 97.5th percentile,16th percentile,84th percentile
Date,,,,,
2010-01-20,-3.244024,-5.656335,-0.646839,-4.369168,-2.172346
2010-01-27,-2.868851,-5.010542,-0.410658,-3.912848,-1.804370
2010-02-08,-2.861758,-5.380910,-0.279860,-3.894311,-1.684651
2010-02-16,-2.706486,-4.915545,-0.506439,-3.715738,-1.702252
2010-02-23,-2.603908,-4.510432,-0.651572,-3.580092,-1.674178
...,...,...,...,...,...
2025-09-09,-0.105894,-0.520619,0.332873,-0.311633,0.107185
2025-09-16,-0.140222,-0.560710,0.299513,-0.339050,0.077109
2025-09-23,-0.069477,-0.479332,0.349763,-0.281311,0.149779


In [62]:
hlb_df = hlb_df[hlb_df.index >= '2024-01-01']
hlb_df


,50th percentile,Elasticity - 2.5th percentile,Elasticity - 97.5th percentile,16th percentile,84th percentile
Date,,,,,
2024-01-05,-0.137023,-0.520891,0.313169,-0.330752,0.074726
2024-01-12,-0.224709,-0.586767,0.146574,-0.404356,-0.034788
2024-01-22,-0.148230,-0.515617,0.282543,-0.334767,0.061439
2024-01-29,-0.130039,-0.519838,0.305060,-0.336219,0.079065
2024-02-08,-0.124572,-0.498897,0.295072,-0.316011,0.075382
...,...,...,...,...,...
2025-09-09,-0.105894,-0.520619,0.332873,-0.311633,0.107185
2025-09-16,-0.140222,-0.560710,0.299513,-0.339050,0.077109
2025-09-23,-0.069477,-0.479332,0.349763,-0.281311,0.149779


5. Now that all our data is ready, make the graph and have it save itself as a .html file to graph_output whenver the notebooks is rerun. The name of the file should exactly match the notebook name. For instance, this file "example.ipynb" produces the graph "example.html." Nice work, you made a graph! 

In [64]:
# First make the figure
fig = go.Figure()


# Loop the columns of the dataframe and plot each as a separate trace
for col in hlb_df.columns:
    if 'Elasticity' not in col:
        fig.add_trace(
            go.Scatter(
                x=hlb_df.index,
                y=hlb_df[col],
                mode='lines',
                name=col,
                line=dict(color='#2A547E' if '50' in col else '#7095cf',
                          dash='dot' if '50' not in col else 'solid')
            )
        )

# Update the titles, using the html tage <sup> for a subtitle 
fig.update_layout(
    title = dict(text = 'Reserve Demand Elasticity<br><sup>Weekly </sup>'),
    xaxis_title=None,
    yaxis_title=None,
)

# This is graph specific, but here we want the y-axis to be percent signs 
fig.update_yaxes(
    tickformat=".2f%"
)

# Again, graph specific, we have a mutliyear series and want tick marks to be years
fig.update_xaxes(
    type='date',
    dtick='M4',
    tickformat='%b %Y',
    title_text=None,
)

fig.update_layout(
    height=450,
    width=800,
        legend=dict(
            orientation='h'
        )
)

ut.add_end_labels(fig = fig, offset=0.05)

fig.update_layout(legend=dict(font=dict(size=20)))

# Show our figure (Dimensions may be off on different screen sizes)
fig.show()

# This should be the same for EVERY GRAPH!
# Save it to the graph_output folder with the name matching the file, as HTML
fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")